# LightGBM Classifier Baseline (Model Comparison) v3

- 모델 종류 체급 비교 목적

평가 (Test set):
- PR-AUC
- Recall (Churn)
- Accuracy
- Confusion Matrix
- Classification Report
- Feature Importance


In [13]:
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', None)


In [14]:
df = pd.read_parquet('kkbox_train_feature_v3.parquet')
df.head()

,msno,city,gender,registered_via,is_churn,num_days_active_w7,total_secs_w7,avg_secs_per_day_w7,std_secs_w7,num_songs_w7,avg_songs_per_day_w7,num_unq_w7,num_25_w7,num_100_w7,short_play_w7,skip_ratio_w7,completion_ratio_w7,short_play_ratio_w7,variety_ratio_w7,num_days_active_w14,total_secs_w14,avg_secs_per_day_w14,std_secs_w14,num_songs_w14,avg_songs_per_day_w14,num_unq_w14,num_25_w14,num_100_w14,short_play_w14,skip_ratio_w14,completion_ratio_w14,short_play_ratio_w14,variety_ratio_w14,num_days_active_w21,total_secs_w21,avg_secs_per_day_w21,std_secs_w21,num_songs_w21,avg_songs_per_day_w21,num_unq_w21,num_25_w21,num_100_w21,short_play_w21,skip_ratio_w21,completion_ratio_w21,short_play_ratio_w21,variety_ratio_w21,num_days_active_w30,total_secs_w30,avg_secs_per_day_w30,std_secs_w30,num_songs_w30,avg_songs_per_day_w30,num_unq_w30,num_25_w30,num_100_w30,short_play_w30,skip_ratio_w30,completion_ratio_w30,short_play_ratio_w30,variety_ratio_w30,secs_trend_w7_w30,secs_trend_w14_w30,days_trend_w7_w14,days_trend_w7_w30,songs_trend_w7_w30,songs_trend_w14_w30,skip_trend_w7_w30,completion_trend_w7_w30,days_since_last_payment,has_ever_paid,days_since_last_cancel,has_ever_cancelled,is_auto_renew_last,last_plan_days,last_payment_method,is_free_user,total_payment_count,total_amount_paid,avg_amount_per_payment,unique_plan_count,subscription_months_est,payment_count_last_30d,payment_count_last_90d,reg_days
0,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,unknown,7,0,7,75448.625000,10778.375000,9128.514648,338,48.285713,159,39,271,54,0.115385,0.801775,0.159763,0.470414,14,177639.296875,12688.521484,10458.754883,842,60.142857,480,127,641,157,0.150831,0.761283,0.186461,0.570071,20,238367.421875,11918.371094,9021.441406,1156,57.799999,663,170,863,220,0.147059,0.746540,0.190311,0.573529,30,358554.000000,11951.799805,7876.637695,1776,59.200001,1040,277,1296,355,0.155968,0.729730,0.199887,0.585586,0.210425,0.495432,0.5,0.233333,0.190315,0.474099,-0.040584,0.072045,5,1,999,0,1,30,41.0,0,1,129,129.0,1,1.0,1,1,2026
1,yLkV2gbZ4GLFwqTOXLVHz0VGrMYcgBGgKZ3kj9RiYu8=,4,male,9,0,6,123668.695312,20611.449219,9505.349609,557,92.833336,67,14,518,22,0.025135,0.929982,0.039497,0.120287,6,123668.695312,20611.449219,9505.349609,557,92.833336,67,14,518,22,0.025135,0.929982,0.039497,0.120287,6,123668.695312,20611.449219,9505.349609,557,92.833336,67,14,518,22,0.025135,0.929982,0.039497,0.120287,6,123668.695312,20611.449219,9505.349609,557,92.833336,67,14,518,22,0.025135,0.929982,0.039497,0.120287,1.000000,1.000000,1.0,1.000000,1.000000,1.000000,0.000000,0.000000,1,1,999,0,1,30,39.0,0,2,298,149.0,1,2.0,1,2,2024
2,I0yFvqMoNkM8ZNHb617e1RBzIS/YRKemHO7Wj13EtA0=,13,male,9,0,3,16989.527344,5663.175781,1434.409424,70,23.333334,65,2,65,3,0.028571,0.928571,0.042857,0.928571,10,50269.140625,5026.914062,3101.173584,249,24.900000,182,43,195,47,0.172691,0.783133,0.188755,0.730924,15,63667.992188,4244.532715,2992.634277,352,23.466667,273,77,206,107,0.218750,0.585227,0.303977,0.775568,18,80453.320312,4469.628906,2823.026123,416,23.111111,337,77,269,108,0.185096,0.646635,0.259615,0.810096,0.211172,0.624824,0.3,0.166667,0.168269,0.598558,-0.156525,0.281937,5,1,999,0,1,30,40.0,0,1,149,149.0,1,1.0,1,1,2022
3,OoDwiKZM+ZGr9P3fRivavgOtglTEaNfWJO4KaJcTTts=,1,unknown,7,1,1,6168.049805,6168.049805,0.000000,23,23.000000,23,0,22,0,0.000000,0.956522,0.000000,1.000000,2,8142.378906,4071.189453,2965.408447,35,17.500000,34,2,30,4,0.057143,0.857143,0.114286,0.971429,2,8142.378906,4071.189453,2965.408447,35,17.500000,34,2,30,4,0.057143,0.857143,0.114286,0.971429,3,8613.391602,2871.130615,2952.498535,38,12.666667,37,3,31,5,0.078947,0.815789,0.131579,0.973684,0.716100,0.945316,0.5,0.333333,0.605263,0.921053,-0.078947,0.140732,6,1,999,0,1,30,41.0,0,1,149,149.0,1,1.0,1,1,2022
4,4De1jAxNRABoyRBDZ82U0yEmzYkqeOugRGVNIf92Xb8=,4,female,9,0,2,5703.128906,2851.564453,2644.321289,29,14.500000,24,5,24,5,0.172414,0.827586,0.172414,0.827586,5,15160.677734,3032.135498,1988.283691,90,18.000000,42,6,82,6,0.066667,0.911111,0.06

In [15]:
RANDOM_STATE = 719

ID_COL = "msno"
TARGET_COL = "is_churn"

CATEGORICAL_COLS = [
    "city", "gender", "registered_via", "last_payment_method",
    "has_ever_paid", "has_ever_cancelled", "is_auto_renew_last", "is_free_user",
]

NUMERICAL_COLS = [
    "reg_days",
    "num_days_active_w7", "total_secs_w7", "avg_secs_per_day_w7", "std_secs_w7",
    "num_songs_w7", "avg_songs_per_day_w7", "num_unq_w7", "num_25_w7", "num_100_w7",
    "short_play_w7", "skip_ratio_w7", "completion_ratio_w7", "short_play_ratio_w7", "variety_ratio_w7",
    "num_days_active_w14", "total_secs_w14", "avg_secs_per_day_w14", "std_secs_w14",
    "num_songs_w14", "avg_songs_per_day_w14", "num_unq_w14", "num_25_w14", "num_100_w14",
    "short_play_w14", "skip_ratio_w14", "completion_ratio_w14", "short_play_ratio_w14", "variety_ratio_w14",
    "num_days_active_w21", "total_secs_w21", "avg_secs_per_day_w21", "std_secs_w21",
    "num_songs_w21", "avg_songs_per_day_w21", "num_unq_w21", "num_25_w21", "num_100_w21",
    "short_play_w21", "skip_ratio_w21", "completion_ratio_w21", "short_play_ratio_w21", "variety_ratio_w21",
    "num_days_active_w30", "total_secs_w30", "avg_secs_per_day_w30", "std_secs_w30",
    "num_songs_w30", "avg_songs_per_day_w30", "num_unq_w30", "num_25_w30", "num_100_w30",
    "short_play_w30", "skip_ratio_w30", "completion_ratio_w30", "short_play_ratio_w30", "variety_ratio_w30",
    "secs_trend_w7_w30", "secs_trend_w14_w30", "days_trend_w7_w14", "days_trend_w7_w30",
    "songs_trend_w7_w30", "songs_trend_w14_w30", "skip_trend_w7_w30", "completion_trend_w7_w30",
    "days_since_last_payment", "days_since_last_cancel", "last_plan_days",
    "total_payment_count", "total_amount_paid", "avg_amount_per_payment",
    "unique_plan_count", "subscription_months_est", "payment_count_last_30d", "payment_count_last_90d",
]

FEATURE_COLS = CATEGORICAL_COLS + NUMERICAL_COLS

X = df[FEATURE_COLS].copy()
y = df[TARGET_COL].astype(int).copy()

## 1. Train / Test Split


In [16]:
from sklearn.model_selection import train_test_split

X = df[FEATURE_COLS]
y = df[TARGET_COL].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=RANDOM_STATE,
    stratify=y,
)

print(X_train.shape, X_test.shape)


(688772, 83) (172194, 83)


## 2. Column Groups


In [17]:
cat_cols = [c for c in CATEGORICAL_COLS if c in X_train.columns]
num_cols = [c for c in NUMERICAL_COLS if c in X_train.columns]

print(f"num_cols: {len(num_cols)}")
print(f"cat_cols: {len(cat_cols)}")


num_cols: 75
cat_cols: 8


## 3. Preprocessing (LightGBM Native Categorical)


In [18]:
cat_feature_indices = [X_train.columns.get_loc(col) for col in cat_cols]

X_train_lgb = X_train.copy()
X_test_lgb = X_test.copy()

for col in cat_cols:
    X_train_lgb[col] = X_train_lgb[col].astype("category")
    X_test_lgb[col] = X_test_lgb[col].astype("category")


## 4. LightGBM Classifier Model


In [19]:
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier(
    n_estimators=3000,
    learning_rate=0.03,
    num_leaves=63,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary",
    class_weight="balanced",
    random_state=RANDOM_STATE,
    n_jobs=-1,
)


## 5. Train Model


In [20]:
lgb_model.fit(
    X_train_lgb,
    y_train,
    eval_set=[(X_test_lgb, y_test)],
    categorical_feature=cat_cols,
    callbacks=[lgb.early_stopping(stopping_rounds=200, verbose=False)],
)


[LightGBM] [Info] Number of positive: 65158, number of negative: 623614
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.225390 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16260
[LightGBM] [Info] Number of data points in the train set: 688772, number of used features: 83
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000


,boosting_type,'gbdt'
,num_leaves,63
,max_depth,-1
,learning_rate,0.03
,n_estimators,3000
,subsample_for_bin,200000
,objective,'binary'
,class_weight,'balanced'
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


## 6. Test Evaluation


In [21]:
import numpy as np
import pandas as pd
from sklearn.metrics import (
    average_precision_score,
    recall_score,
    accuracy_score,
    confusion_matrix,
    classification_report,
)

y_proba = lgb_model.predict_proba(X_test_lgb)[:, 1]
y_pred  = lgb_model.predict(X_test_lgb)

print(f"PR-AUC: {average_precision_score(y_test, y_proba):.4f}")
print(f"Recall (Churn): {recall_score(y_test, y_pred):.4f}")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")


PR-AUC: 0.9339
Recall (Churn): 0.9019
Accuracy: 0.9710


## 7. Confusion Matrix


In [22]:
cm = confusion_matrix(y_test, y_pred)
pd.DataFrame(cm, index=["Actual 0", "Actual 1"], columns=["Pred 0", "Pred 1"])


,Pred 0,Pred 1
Actual 0,152500,3404
Actual 1,1598,14692


## 8. Classification Report


In [23]:
print(classification_report(y_test, y_pred, digits=4))


              precision    recall  f1-score   support

           0     0.9896    0.9782    0.9839    155904
           1     0.8119    0.9019    0.8545     16290

    accuracy                         0.9710    172194
   macro avg     0.9008    0.9400    0.9192    172194
weighted avg     0.9728    0.9710    0.9716    172194



## 9. Feature Importance (Gain)


In [24]:
feature_names = X_train_lgb.columns
importances = lgb_model.feature_importances_

imp_lgb_df = pd.DataFrame({
    "feature": feature_names,
    "importance": importances,
}).sort_values("importance", ascending=False)

imp_lgb_df


,feature,importance
8,reg_days,8650
73,days_since_last_payment,7391
72,completion_trend_w7_w30,4533
0,city,4299
64,variety_ratio_w30,4277
71,skip_trend_w7_w30,4230
74,days_since_last_cancel,3861
22,variety_ratio_w7,3656
66,secs_trend_w14_w30,3613
50,variety_ratio_w21,3481
